<a href="https://colab.research.google.com/github/anismousse/ml-class/blob/master/02_cifar10_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

# First try 
basic sequential network

In [2]:
import tensorflow as tf
import wandb

wandb.init()

config = wandb.config
config.batch_size = 128
config.epochs = 10
config.learn_rate = 0.001

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(class_names)

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255. # 255 is coming from RGB

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=X_train.shape[1:])) # shaping in ne line each images this is a deterministique leyaer nothing happen here in term of learning
model.add(tf.keras.layers.Dense(100 , activation='relu')) # this is retriving 100 number between 0 and 1
model.add(tf.keras.layers.Dense(num_classes , activation='softmax')) # here we do have or this case 10 nurons
# until this point we do have 3073- free paramaters= 32*32*3*10 +10(biaises)
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(config.learn_rate),
              metrics=['accuracy'])
# log the number of total parameters
config.total_params = model.count_params()
print("Total params: ", config.total_params)
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test),
          callbacks=[wandb.keras.WandbCallback(data_type="image", labels=class_names, save_model=False)])


<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


170500096/170498071 [==============================] - 4s 0us/step
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Total params:  308310
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 5s 93us/sample - loss: 1.9219 - acc: 0.3148 - val_loss: 1.8093 - val_acc: 0.3535
Epoch 2/10
50000/50000 [==============================] - 4s 83us/sample - loss: 1.7743 - acc: 0.3745 - val_loss: 1.7408 - val_acc: 0.3870
Epoch 3/10
50000/50000 [==============================] - 4s 80us/sample - loss: 1.7052 - acc: 0.3988 - val_loss: 1.6805 - val_acc: 0.4076
Epoch 4/10
50000/50000 [==============================] - 4s 85us/sample - loss: 1.6575 - acc: 0.4122 - val_loss: 1.6547 - val_acc: 0.4143
Epoch 5/10
50000/50000 [==============================] - 4s 81us/sample - loss: 1.6266 - acc: 0.4236 - val_loss: 1.6694 - val_acc: 0.4141
Epoch 6/10
50000/50000 [========================

# Second take
Adding a convolution


In [3]:
import tensorflow as tf
import wandb

# Set Hyper-parameters
wandb.init()
config = wandb.config
config.batch_size = 128
config.epochs = 10
config.learn_rate = 0.001
config.dropout = 0.3
config.dense_layer_nodes = 128

# Load data
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(class_names)

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Define model
model = tf.keras.models.Sequential() #32*32*3
model.add(tf.keras.layers.Conv2D(16, (3, 3), padding='same',
                                 input_shape=X_train.shape[1:], activation='relu')) # 25 convolution  the output 32*32*25 // 16output image
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) # out 16*16*25  (16 is comng for the 2^x as max pooling is a dimension rduction operation)
model.add(tf.keras.layers.Dropout(config.dropout))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(config.dense_layer_nodes, activation='relu'))
model.add(tf.keras.layers.Dropout(config.dropout))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(config.learn_rate),
              metrics=['accuracy'])
# log the number of total parameters
config.total_params = model.count_params()
print("Total params: ", config.total_params)

model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test),
          callbacks=[wandb.keras.WandbCallback(data_type="image", labels=class_names, save_model=False)])


Total params:  526154
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 32s 636us/sample - loss: 1.7816 - acc: 0.3494 - val_loss: 1.4795 - val_acc: 0.4760
Epoch 2/10
50000/50000 [==============================] - 31s 630us/sample - loss: 1.4979 - acc: 0.4572 - val_loss: 1.3515 - val_acc: 0.5255
Epoch 3/10
50000/50000 [==============================] - 31s 626us/sample - loss: 1.4122 - acc: 0.4920 - val_loss: 1.2791 - val_acc: 0.5494
Epoch 4/10
50000/50000 [==============================] - 31s 626us/sample - loss: 1.3540 - acc: 0.5136 - val_loss: 1.2348 - val_acc: 0.5649
Epoch 5/10
50000/50000 [==============================] - 31s 625us/sample - loss: 1.3136 - acc: 0.5267 - val_loss: 1.2119 - val_acc: 0.5751
Epoch 6/10
50000/50000 [==============================] - 31s 625us/sample - loss: 1.2801 - acc: 0.5397 - val_loss: 1.1707 - val_acc: 0.5794
Epoch 7/10
50000/50000 [==============================] - 31s 625us/sample - loss:

# Third take
multiples convolution and image preprocessing

In [0]:
import tensorflow as tf
import wandb

# Set Hyper-parameters
wandb.init()
config = wandb.config
config.batch_size = 128
config.epochs = 10
config.learn_rate = 0.001
config.dropout = 0.3
config.dense_layer_nodes = 128

# Load data
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(class_names)

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
#print(y_train.shape)
# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

#print(y_train.shape)
#exit()
# Define model
model = tf.keras.models.Sequential() #32*32*3
model.add(tf.keras.layers.Conv2D(10, (3, 3), padding='same',input_shape=X_train.shape[1:], activation='relu')) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) 
model.add(tf.keras.layers.Dropout(0.3))
#model.add(tf.keras.layers.Conv2D(20, (3, 3), padding='same', activation='relu')) 
#model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) 
#model.add(tf.keras.layers.Dropout(0.3))
#model.add(tf.keras.layers.Conv2D(40, (3, 3), padding='same', activation='relu')) 
#model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) 
model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(config.learn_rate),
              metrics=['accuracy'])
# log the number of total parameters
config.total_params = model.count_params()
print("Total params: ", config.total_params)

model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test),
          callbacks=[wandb.keras.WandbCallback(data_type="image", labels=class_names, save_model=False)])


Total params:  257390
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 27s 537us/sample - loss: 1.6244 - acc: 0.4212 - val_loss: 1.4225 - val_acc: 0.4851
Epoch 2/10
50000/50000 [==============================] - 27s 533us/sample - loss: 1.3495 - acc: 0.5210 - val_loss: 1.2841 - val_acc: 0.5445
Epoch 3/10
50000/50000 [==============================] - 27s 537us/sample - loss: 1.2484 - acc: 0.5597 - val_loss: 1.1930 - val_acc: 0.5805
Epoch 4/10
 9728/50000 [====>.........................] - ETA: 19s - loss: 1.1738 - acc: 0.5887